In [ ]:
# Carreguem les dades
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
data = pd.read_csv("dades.csv", decimal = ',')
nFiltres =48
nf = nFiltres+1
filtres = {}
#Display de totes les dades en print
#pd.set_option('display.max_rows', filtres["filtre1"].shape[0]+1)
# creem DF
Terb = data["TAG02TI01803_ATI"]
for i in range(1,nf):
    filtres["filtre{}".format(i)] = pd.DataFrame()
    OV = data["TAG02MV005{}_POS".format(str(i).zfill(2))]
    Q = data["TAG02FT005{}_CI".format(str(i).zfill(2))]
    AP = data["TAG02PDT05{}_PRS".format(str(i).zfill(2))]
    filtres["filtre{}".format(i)]["OV"] = OV[::-1]
    filtres["filtre{}".format(i)]["Q"] = Q[::-1]
    filtres["filtre{}".format(i)]["AP"] = AP[::-1]
    filtres["filtre{}".format(i)]["Terb"] = Terb[::-1]
for filtre in filtres:
    filtres[filtre] = filtres[filtre].reset_index()
    
# Estat
# Creem una variable "estat" on definirem l'estat del filtre


OVmin=20
Qmin = 20
DifAP = 300
APmin = 10
APmincic = 300
for filtre in filtres:
    filtres[filtre]["Estat"] = pd.Series()
    filtres[filtre]["cicle"] = pd.Series()
    filtres[filtre].loc[0,"cicle"] = 1
    for m in range(1,len(filtres[filtre]["AP"])):
        if filtres[filtre]["AP"][m-1] - filtres[filtre]["AP"][m] > DifAP:
            filtres[filtre].loc[m,"Estat"] = "Tancat"
        elif filtres[filtre]["Q"][m] < Qmin:
            filtres[filtre].loc[m,"Estat"] = "Tancat"
        elif filtres[filtre]["OV"][m] < OVmin:
            filtres[filtre].loc[m,"Estat"] = "Tancat"
        elif filtres[filtre]["AP"][m] < APmin:
            filtres[filtre].loc[m,"Estat"] = "Tancat"
        else: 
            filtres[filtre].loc[m,"Estat"] = "Actiu"
#Definim la variable cicle per a filtres
for filtre in filtres:
    filtres[filtre]["cicle"] = pd.Series()
    filtres[filtre].loc[0,"cicle"] = 1
    for i in range(1,len(filtres[filtre]["cicle"])):
        if filtres[filtre]["Estat"][i] == "Actiu"\
        and filtres[filtre]["Estat"][i-1] == "Tancat"\
        and filtres[filtre]["AP"][i] < APmincic:
            filtres[filtre].loc[i,"cicle"] = filtres[filtre]["cicle"][i-1] + 1
        else:
            filtres[filtre].loc[i,"cicle"] = filtres[filtre]["cicle"][i-1]
#Treiem ultim cicle
for filtre in filtres:
    for i in range(0,len(filtres[filtre]["cicle"])):
        if filtres[filtre]["cicle"][i] == 1:
            filtres[filtre] = filtres[filtre].drop([i])
        elif filtres[filtre]["cicle"][i] == max(filtres[filtre]["cicle"]):
            filtres[filtre] = filtres[filtre].drop([i])
    filtres[filtre] = filtres[filtre].reset_index()
# Redefinim els cicles
for filtre in filtres:
    for i in range(0,len(filtres[filtre]["cicle"])):
        filtres[filtre].loc[i,"cicle"] = filtres[filtre]["cicle"][i] - 1

        # Corretgim unitats cabal
for f in filtres:
    for k in range(0,len(filtres[f]["Q"])):
        filtres[f].loc[k,"Q"] = filtres[f]["Q"][k]*3600 / 1000
#Unitats Cabal: m3/s

# Creem VolumFiltrat
for f in filtres:
    filtres[f]["Vfiltrat"] = pd.Series()
    for k in range(0,len(filtres[f]["Vfiltrat"])):
        if k <2:
            filtres[f].loc[k,"Vfiltrat"] = filtres[f]["Q"][k]
        elif filtres[f]["Estat"][k] == "Tancat":
            filtres[f].loc[k,"Vfiltrat"] = 0
        else:
            filtres[f].loc[k,"Vfiltrat"] = filtres[f]["Vfiltrat"][k-1] + filtres[f]["Q"][k]

# Creem Hores Totals Filtrades
for f in filtres:
    filtres[f]["HTF"] = pd.Series()
    filtres[f].loc[0,"HTF"]=0
    for k in range(1,len(filtres[f]["HTF"])):
        if filtres[f]["Estat"][k-1] =="Tancat" and filtres[f]["Estat"][k] == "Actiu":
            filtres[f].loc[k,"HTF"] = 1
        if filtres[f]["Estat"][k]=="Actiu" and filtres[f]["Estat"][k-1] == "Actiu":
            filtres[f].loc[k,"HTF"] = filtres[f]["HTF"][k-1] +1
        else:
            filtres[f].loc[k,"HTF"] = 0
# TEMPS TOTAL FILTRAT: TTF

for f in filtres:
    filtres[f]["TTF"] = pd.Series()
    filtres[f].loc[0,"TTF"] = 1
    for i in range(1,len(filtres[f]["TTF"])):
        if filtres[f]["Estat"][i] == "Actiu":
            filtres[f].loc[i,"TTF"] = filtres[f]["TTF"][i-1]+1
        else:
            filtres[f].loc[i,"TTF"] = filtres[f]["TTF"][i-1]
            
#TerbAcTotal
for f in filtres:
    filtres[f]["TerbAcTotal"] = pd.Series()
    filtres[f].loc[0,"TerbAcTotal"] = 0
    for i in range(1,len(filtres[f]["TerbAcTotal"])):
        if filtres[f]["Estat"][i] == "Actiu":
            filtres[f].loc[i,"TerbAcTotal"] = filtres[f]["TerbAcTotal"][i-1] \
            + filtres[f]["Terb"][i]
        else:
            filtres[f].loc[i,"TerbAcTotal"] = filtres[f]["TerbAcTotal"][i-1]

#Treiem filtres Tancats
# Treure files on el filtre està aturat
for f in filtres:
    for m in range(0, len(filtres[f]["Estat"])):
        if filtres[f]["Estat"][m] == "Tancat":
            filtres[f] = filtres[f].drop([m])
    filtres[f] = filtres[f].reset_index(drop=True)
### Treiem infinits i anomals
for filtre in filtres:
    filtres[filtre] = filtres[filtre].replace([np.inf, -np.inf], np.nan)
    filtres[filtre] = filtres[filtre].dropna()
    filtres[filtre] = filtres[filtre].reset_index(drop=True)

### Treure filtres
filtres_lookup = dict(filtres)
for filtre in filtres_lookup:
    if sum(filtres[filtre]["AP"]) == 0:
        del filtres[filtre]
    elif sum(filtres[filtre]["Q"])== 0:
        del filtres[filtre]
    elif sum(filtres[filtre]["OV"]) == 0:
        del filtres[filtre]
        
# Creem AP:Q
for f in filtres:
    filtres[f]["AP:Q"] = pd.Series()
    for k in range(0,len(filtres[f]["AP:Q"])):
        filtres[f].loc[k,"AP:Q"] = filtres[f]["AP"][k] / filtres[f]["Q"][k]
# Creem AP:VTF
for f in filtres:
    filtres[f]["AP:VTF"] = pd.Series()
    for k in range(0,len(filtres[f]["AP:VTF"])):
        filtres[f].loc[k,"AP:VTF"] = filtres[f]["AP"][k] / filtres[f]["Vfiltrat"][k]
        
# Agrupar i desar dades
tot = pd.DataFrame()
for f in filtres:
    filtres[f]["Filtre"] = pd.Series()
    for k in range(0,len(filtres[f]["Filtre"])):
        filtres[f].loc[k,"Filtre"] = "{}".format(f)

for f in filtres:
    tot = tot.append(filtres[f],sort=False)
    tot = tot.reset_index(drop=True)

tot.to_pickle("data")

# Carregar dades
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_pickle("data")
nFiltres =48
nf = nFiltres+1

filtres = {}
for i in range(1,nf):
    filtres["filtre{}".format(i)] = pd.DataFrame()

for f in filtres:
    for k in range(0,len(data)):
        if data["Filtre"][k] == "{}".format(f):
            filtres[f] = filtres[f].append(data.loc[k],ignore_index=True)
            
# Eliminar filtres buits de dades

filtres_lookup = filtres.copy()
for f in filtres_lookup:
    if len(filtres[f]) == 0:
        del filtres[f]
# Netejar AP=0
for f in filtres:
    for m in range(0, len(filtres[f]["AP"])):
        if filtres[f]["AP"][m] == 0:
            filtres[f] = filtres[f].drop([m])
            
# Correcció index de DF

for f in filtres:
    filtres[f]= filtres[f].reset_index(drop=True)

for f in filtres:
    filtres[f]["VF2"] = pd.Series()
    for k in range(0,len(filtres[f]["Vfiltrat"])):
        filtres[f].loc[k,"VF2"] = filtres[f]["Vfiltrat"][k]*filtres[f]["Vfiltrat"][k]
# Neteja de la matriu de dades. Aquestes variables que s'esborren en  les seguents linies de 
# codi s'han estudiat, però s'ha vist que no aporten informació rellevant a l'estudi
for f in filtres:
    del filtres[f]["level_0"]
    del filtres[f]["Terb"]
    del filtres[f]["TerbAcTotal"]
    del filtres[f]["AP:VTF"]
    del filtres[f]["index"]
    
# Creació matriu de dades ANS
dfNS = pd.DataFrame(columns=["HTF","VTF","β0","β1","filtre","colorB0","colorB1","colorVTF","colorHTF"])
count = 0
for f in filtres:
    dfNS.loc[count,"filtre"]= "{}".format(f)
    count = count+1
    
dfNS = dfNS.set_index("filtre")

# Regressió lineal per cada filtre
# Regressió
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn import linear_model

coeficientsR2 = {}
dfNS["R2"] = pd.Series()
regr = linear_model.LinearRegression()
for f in filtres:
    filtres[f]["APe"] = pd.Series()
    x = filtres[f]["Vfiltrat"]
    y = filtres[f]["AP:Q"]
    mymodel = np.poly1d(np.polyfit(x,y,1))
    dfNS.loc[f,"β0"] = mymodel[0]
    dfNS.loc[f,"β1"] = mymodel[1]
    dfNS.loc[f,"R2"] = r2_score(y, mymodel(x))
#a = regr.predict(filtres["filtre5"]["HTF"])
#    if f == "filtre5":
#        plt.scatter(filtres["filtre5"]["HTF"],filtres["filtre5"]["AP:Q"],color="black")
#        plt.plot(filtres["filtre5"]["HTF"],regr.predict(filtres["filtre5"]["HTF"]))

# Valors mitjans

for f in filtres:
    countVTF=0
    countHTF=0
    for m in range(1,len(filtres[f])):
        if filtres[f]["Vfiltrat"][m] < filtres[f]["Vfiltrat"][m-1]:
            countVTF = countVTF + filtres[f]["Vfiltrat"][m-1]
        if filtres[f]["HTF"][m] < filtres[f]["HTF"][m-1]:
            countHTF = countHTF + filtres[f]["HTF"][m-1]
        dfNS.loc[f,"HTF"] = countHTF/max(filtres[f]["cicle"])
        dfNS.loc[f,"VTF"] = countVTF/max(filtres[f]["cicle"])
            
# nº de cicles a matriu AND
dfNS["cicles"]=pd.Series()

for f in filtres:
    dfNS.loc[f,"cicles"]= max(filtres[f]["cicle"]
                              
# Clustering 
from sklearn.cluster import KMeans
####### b0
means = KMeans(n_clusters=3,random_state=3425)
cB0 = means.fit_predict(dfNS[["β0"]])
ord_idx=np.argsort(means.cluster_centers_.flatten())
cntrs = np.zeros_like(cB0)-1
for i in np.arange(3):
    cntrs[cB0==ord_idx[i]]=i
dfNS["numB0"] = cntrs
centroidsB0 = means.cluster_centers_
####### b1
means = KMeans(n_clusters=3,random_state=3425)
cB1 = means.fit_predict(dfNS[["β1"]])
ord_idx=np.argsort(means.cluster_centers_.flatten())
cntrs = np.zeros_like(cB1)-1
for i in np.arange(3):
    cntrs[cB1==ord_idx[i]]=i
dfNS["numB1"] = cntrs
centroidsB1 = means.cluster_centers_

####### vtf
means = KMeans(n_clusters=3,random_state=3425)
cVTF = means.fit_predict(dfNS[["VTF"]])
ord_idx=np.argsort(means.cluster_centers_.flatten())
cntrs = np.zeros_like(cVTF)-1
for i in np.arange(3):
    cntrs[cVTF==ord_idx[i]]=i
dfNS["numVTF"] = cntrs
centroidsVTF = means.cluster_centers_

####### htf
means = KMeans(n_clusters=3,random_state=3425)
cHTF = means.fit_predict(dfNS[["HTF"]])
ord_idx=np.argsort(means.cluster_centers_.flatten())
cntrs = np.zeros_like(cHTF)-1
for i in np.arange(3):
    cntrs[cHTF==ord_idx[i]]=i
dfNS["numHTF"] = cntrs
centroidsHTF = means.cluster_centers_
                        
# Gama de colors per representar KPI user friendly. Aquí's'assigna el color de la "performance" 
# als diferents clusters.
pintura = pd.DataFrame()
pintura["f"] = pd.Series("filtre{}".format(f) for f in range(1,nf))
pintura = pintura.set_index("f")

pintura["B0"] = pd.Series(dfNS["numB0"])
pintura["B1"] = pd.Series(dfNS["numB1"])
pintura["HTF"] = pd.Series(dfNS["numHTF"])
pintura["VTF"] = pd.Series(dfNS["numVTF"])

pintura["colorB0"] = pd.Series()
pintura["colorB1"] = pd.Series()
pintura["colorHTF"] = pd.Series()
pintura["colorVTF"] = pd.Series()

for k in pintura.index:
    if pintura["B0"][k] == 0:
        pintura.loc[k,"colorB0"] = "lime"
    elif pintura["B0"][k] == 1:
        pintura.loc[k,"colorB0"] = "gold"
    elif pintura["B0"][k] == 2:
        pintura.loc[k,"colorB0"] = "red"
    else:
        pintura.loc[k,"colorB0"] = "white"
        
for k in pintura.index:
    if pintura["B1"][k] == 0:
        pintura.loc[k,"colorB1"] = "lime"
    elif pintura["B1"][k] == 1:
        pintura.loc[k,"colorB1"] = "gold"
    elif pintura["B1"][k] == 2:
        pintura.loc[k,"colorB1"] = "red"
    else:
        pintura.loc[k,"colorB1"] = "white"
    
for k in pintura.index:
    if pintura["HTF"][k] == 0:
        pintura.loc[k,"colorHTF"] = "red"
    elif pintura["HTF"][k] == 1:
        pintura.loc[k,"colorHTF"] = "gold"
    elif pintura["HTF"][k] == 2:
        pintura.loc[k,"colorHTF"] = "lime"
    else:
        pintura.loc[k,"colorHTF"] = "white"
    
for k in pintura.index:
    if pintura["VTF"][k] == 0:
        pintura.loc[k,"colorVTF"] = "red"
    elif pintura["VTF"][k] == 1:
        pintura.loc[k,"colorVTF"] = "gold"
    elif pintura["VTF"][k] == 2:
        pintura.loc[k,"colorVTF"] = "lime"
    else:
        pintura.loc[k,"colorVTF"] = "white"

# Fins la linia 390: aquest codi reestructura les dades per tal que la representació
# gràica s'ajusti a la disposició espacial real dels filtres
estructura_colors = pd.DataFrame()
txt = pd.Series("F{}".format(i) for i in range(1,nf))
estructura_colors["txt"] = txt
estructura_colors["ordre"] = pd.Series()
estructura_colors.loc[0,"ordre"] = 3
estructura_colors.loc[1,"ordre"] = 2
estructura_colors.loc[2,"ordre"] = 1
estructura_colors.loc[3,"ordre"] = 0
estructura_colors = estructura_colors.fillna(0)
estructura_colors["ordre"] = estructura_colors["ordre"]+1
for i in range(4,47):
    estructura_colors.loc[i,"ordre"] = estructura_colors["ordre"][i-4] + 4
    
for i in range(0,len(estructura_colors)):
    estructura_colors.loc[i,"ordre"] =estructura_colors["ordre"][i]-1
estructura_colors.loc[47,"ordre"] = 44

for i in range(0,len(estructura_colors)):
    estructura_colors.loc[estructura_colors["ordre"][i],"B0"] = pintura["colorB0"][i]
    estructura_colors.loc[estructura_colors["ordre"][i],"B1"] = pintura["colorB1"][i]
    estructura_colors.loc[estructura_colors["ordre"][i],"HTF"] = pintura["colorHTF"][i]
    estructura_colors.loc[estructura_colors["ordre"][i],"VTF"] = pintura["colorVTF"][i]

# Representació de variables
from matplotlib import pyplot as plt
import random
x = [i // 4 for i in range(48)]
y = [i % 4 for i in range(48)]
z = [i for i in estructura_colors["B0"].tolist()]
# create a scatter plot for the given x,y positions, make them quite large, color them
# using z and the Red-Yellow-Green color map, give them a black border 
plt.scatter(x, y , c = estructura_colors["B0"].to_list(),s=400, ls='-', edgecolors='black', \
           marker = "s",linewidths = 1)
#plt.colorbar() #fiqem barra de colors
plt.xlim(-0.9, 12) # limits
plt.ylim(-0.9, 3.9)
plt.title('Agrupacions per β0')
plt.grid(False)
plt.axis("On")
ax = plt.axes()
ax.yaxis.set_major_locator(plt.NullLocator())
ax.xaxis.set_major_locator(plt.NullLocator())

txt = pd.Series("F{}".format(i) for i in range(1,nf))
    # LABELS NOM
count = 0
for i in range(0,12):
    a = plt.annotate(txt.to_list()[i*4],xy=(-0.3+i,3.4),weight="bold") 
    b = plt.annotate(txt.to_list()[i*4+1],xy=(-0.3+i,2.4),weight="bold") 
    c = plt.annotate(txt.to_list()[i*4+2],xy=(-0.3+i,1.4),weight="bold") 
    d = plt.annotate(txt.to_list()[i*4+3],xy=(-0.3+i,0.4),weight="bold") 
    a.set_fontsize(8)
    b.set_fontsize(8)
    c.set_fontsize(8)
    d.set_fontsize(8)
plt.show() 
from matplotlib import pyplot as plt
import random
x = [i // 4 for i in range(48)]
y = [i % 4 for i in range(48)]
z = [i for i in estructura_colors["B1"].tolist()]
# create a scatter plot for the given x,y positions, make them quite large, color them
# using z and the Red-Yellow-Green color map, give them a black border 
plt.scatter(x, y , c = estructura_colors["B1"].to_list(),s=400, ls='-', edgecolors='black', \
           marker = "s",linewidths = 1)
#plt.colorbar() #fiqem barra de colors
plt.xlim(-0.9, 12) # limits
plt.ylim(-0.9, 3.9)
plt.title('Agrupacions per β1')
plt.grid(False)
plt.axis("On")
ax = plt.axes()
ax.yaxis.set_major_locator(plt.NullLocator())
ax.xaxis.set_major_locator(plt.NullLocator())
txt = pd.Series("F{}".format(i) for i in range(1,nf))
    # LABELS NOM
count = 0
for i in range(0,12):
    a = plt.annotate(txt.to_list()[i*4],xy=(-0.3+i,3.4),weight="bold") 
    b = plt.annotate(txt.to_list()[i*4+1],xy=(-0.3+i,2.4),weight="bold") 
    c = plt.annotate(txt.to_list()[i*4+2],xy=(-0.3+i,1.4),weight="bold") 
    d = plt.annotate(txt.to_list()[i*4+3],xy=(-0.3+i,0.4),weight="bold") 
    a.set_fontsize(8)
    b.set_fontsize(8)
    c.set_fontsize(8)
    d.set_fontsize(8)
plt.show() 
from matplotlib import pyplot as plt
import random
x = [i // 4 for i in range(48)]
y = [i % 4 for i in range(48)]
z = [i for i in estructura_colors["VTF"].tolist()]
# create a scatter plot for the given x,y positions, make them quite large, color them
# using z and the Red-Yellow-Green color map, give them a black border 
plt.scatter(x, y , c = estructura_colors["VTF"].to_list(),s=400, ls='-', edgecolors='black', \
           marker = "s",linewidths = 1)
#plt.colorbar() #fiqem barra de colors
plt.xlim(-0.9, 12) # limits
plt.ylim(-0.9, 3.9)
plt.title('Agrupacions per VTF')
plt.grid(False)
plt.axis("On")
ax = plt.axes()
ax.yaxis.set_major_locator(plt.NullLocator())
ax.xaxis.set_major_locator(plt.NullLocator())
txt = pd.Series("F{}".format(i) for i in range(1,nf))
    # LABELS NOM
count = 0
for i in range(0,12):
    a = plt.annotate(txt.to_list()[i*4],xy=(-0.3+i,3.4),weight="bold") 
    b = plt.annotate(txt.to_list()[i*4+1],xy=(-0.3+i,2.4),weight="bold") 
    c = plt.annotate(txt.to_list()[i*4+2],xy=(-0.3+i,1.4),weight="bold") 
    d = plt.annotate(txt.to_list()[i*4+3],xy=(-0.3+i,0.4),weight="bold") 
    a.set_fontsize(8)
    b.set_fontsize(8)
    c.set_fontsize(8)
    d.set_fontsize(8)
plt.show() 
from matplotlib import pyplot as plt
import random
x = [i // 4 for i in range(48)]
y = [i % 4 for i in range(48)]
z = [i for i in estructura_colors["HTF"].tolist()]
# create a scatter plot for the given x,y positions, make them quite large, color them
# using z and the Red-Yellow-Green color map, give them a black border 
plt.scatter(x, y , c = estructura_colors["HTF"].to_list(),s=400, ls='-', edgecolors='black', \
           marker = "s",linewidths = 1)
#plt.colorbar() #fiqem barra de colors
plt.xlim(-0.9, 12) # limits
plt.ylim(-0.9, 3.9)
plt.title('Agrupacions per HTF')
plt.grid(False)
plt.axis("On")
ax = plt.axes()
ax.yaxis.set_major_locator(plt.NullLocator())
ax.xaxis.set_major_locator(plt.NullLocator())
txt = pd.Series("F{}".format(i) for i in range(1,nf))
    # LABELS NOM
count = 0
for i in range(0,12):
    a = plt.annotate(txt.to_list()[i*4],xy=(-0.3+i,3.4),weight="bold") 
    b = plt.annotate(txt.to_list()[i*4+1],xy=(-0.3+i,2.4),weight="bold") 
    c = plt.annotate(txt.to_list()[i*4+2],xy=(-0.3+i,1.4),weight="bold") 
    d = plt.annotate(txt.to_list()[i*4+3],xy=(-0.3+i,0.4),weight="bold") 
    a.set_fontsize(8)
    b.set_fontsize(8)
    c.set_fontsize(8)
    d.set_fontsize(8)
plt.show() 
                              
# ESTUDI ANOVA I DIFERÈNCIES SIGNIFICATIVES
                              
import scipy.stats as stats

VTFadaptat = []
VTFreactivat = []
for f in filtres:
    if dfNS["Carbó"][f] == "Adaptat":
        VTFadaptat.append("{}".format(dfNS["VTF"][f]))
    else:
        VTFreactivat.append("{}".format(dfNS["VTF"][f]))
stats.f_oneway(VTFreactivat,VTFadaptat)
# tEST DE NORMALITAT: S'HA ESTUDIAT LA NORMALITAT DE TOTES LES VARIABLES CANVIANT
# EL PARÀMETRE DE LA FUNCIÓ:
stats.shapiro(dfNS["VTF"])
                              
# Test de levene: s'ha estudiant la homogeneitat de variàncies canviant els
# paràmetre de la funció: 
stats.levene(np.array(B0adaptat).astype(np.float), np.array(B0reactivat).astype(np.float))

# KRUSKAL WALLIS
import scipy.stats as stats

B0adaptat = []
B0reactivat = []
for f in filtres:
    if dfNS["Carbó"][f] == "Adaptat":
        B0adaptat.append("{}".format(dfNS["β0"][f]))
    else:
        B0reactivat.append("{}".format(dfNS["β0"][f]))
stats.kruskal(B0adaptat, B0reactivat)
               
import scipy.stats as stats

B1adaptat = []
B1reactivat = []
for f in filtres:
    if dfNS["Carbó"][f] == "Adaptat":
        B1adaptat.append("{}".format(dfNS["β1"][f]))
    else:
        B1reactivat.append("{}".format(dfNS["β1"][f]))
stats.kruskal(B1adaptat, B1reactivat)
